In [1]:
import _misc
import os
import _myLogging
import xlwings as xw

############### CONNECTION TYPES ################
# 0: EXCEL - called from Excel   ################
# 1: EXCEL - called from Python  ################
# 2: Panel Server                ################
# 3: Panel Local                 ################
# Overrides setting to 0 if called from Excel ###
#################################################

CONNECTIONTYPE=1   #Set to 1 to call Excel from Python (for testing/editing), 2 for deployed Panel Server, 3 for local Panel
MYLOGGER = _myLogging.get_logger("PanelVersion")
MODELTYPE='Experience Rating Tool'

def main():
    global CONNECTIONTYPE,BOOK,SOURCEFILE
    # Check if called from Excel
    try:
        BOOK=xw.Book.caller()
        SOURCEFILE=BOOK.fullname 
        CONNECTIONTYPE=0
    except:
        if CONNECTIONTYPE == 0:
            CONNECTIONTYPE=1  #Override to 1 if not called from Excel

    # Import appropriate modules based on connection type
    if CONNECTIONTYPE in [0,1]:
        # Called from Excel
        import modelAnalysis as Analysis
        import modelFunctions as aFns

        def RunAnalysis(modeltype,book,sourcefile):
            analysis= Analysis.Analysis(CONNECTIONTYPE,modeltype,book,sourcefile)
            
            if len(analysis.error)>0:
                analysis.book.sheets["Navigation"].range("runstatus").value="Model Update Failed"
                _misc.showMessageBox('Error',analysis.error)
            else:
                try:
                    #Include function in modelFunctions.py for any model specific analysis steps
                    aFns.modelSpecificAnalysisSteps(analysis)
                except:
                    pass    

                analysis.book.sheets["Navigation"].range("runstatus").value="Model Update Successful"
                _misc.showMessageBox('Status','Model Run Complete.  Data Model will update after OK.')      

        if CONNECTIONTYPE == 0:
            # Called from Excel
            RunAnalysis(MODELTYPE,BOOK,SOURCEFILE)
        elif CONNECTIONTYPE == 1:
            # Open dialog to select Excel file
            SOURCEFILE = _misc.selectAnalysisFile_LocalVersion()
            if SOURCEFILE=='No File Selected':
                return
            else:
                if(os.path.splitext(SOURCEFILE)[1] in ['.xlsx','.xlsm']):  #If excel file, link with xlwings
                    BOOK=xw.Book(SOURCEFILE) 
                    RunAnalysis(MODELTYPE,BOOK,SOURCEFILE)
                else:  #define what to do if glob
                    RunAnalysis(MODELTYPE,BOOK,SOURCEFILE)
    elif CONNECTIONTYPE== 2:
        # Called from Panel
        import modelPanelSetup as aPanel
        import panel as pn

        MAX_SIZE_MB = 150
        pn.extension(nthreads=10)

        def get_panel(userid,toolname):
            #Panel requires connection type because if server version, requires gzip instead of xlsx/xlsm
            #If local version, either gzip or xlsm/xlsx can be used
            return aPanel.Panel(CONNECTIONTYPE, toolname, userid) 

        ROUTES = {
            "A": get_panel("A",MODELTYPE).view(), "B": get_panel("B",MODELTYPE).view(), "C": get_panel("C",MODELTYPE).view(),
            "D": get_panel("D",MODELTYPE).view(), "E": get_panel("E",MODELTYPE).view(), "F": get_panel("F",MODELTYPE).view(),
            "G": get_panel("G",MODELTYPE).view(), "H": get_panel("H",MODELTYPE).view(), "I": get_panel("I",MODELTYPE).view(),
            "J": get_panel("J",MODELTYPE).view(),
        }

        pn.serve(ROUTES, port=8000, websocket_origin= '*', 
            # Increase the maximum websocket message size allowed by Bokeh
            websocket_max_message_size=MAX_SIZE_MB*1024*1024,
            # Increase the maximum buffer size allowed by Tornado
            http_server_kwargs={'max_buffer_size': MAX_SIZE_MB*1024*1024})
        return aPanel.Panel(CONNECTIONTYPE, MODELTYPE)
    elif CONNECTIONTYPE == 3:
        # Called from Panel
        import modelPanelSetup as aPanel
        import panel as pn
        return aPanel.Panel(CONNECTIONTYPE, MODELTYPE,"none")
if __name__ == "__main__":
    thisPanel=main()

C:\Actuarial Tools\Python-Based Models\Experience-Rating config.ini
Error Creating Config: C:\Actuarial Tools\Python-Based Models\Experience-Rating\config.ini


TypeError: string indices must be integers, not 'str'

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import _cm
df = thisPanel.analysis.preppedspecs['Source Data Formats']

def color_col(col, pattern_map, default=''):
    return np.select(
        [col.str.contains(k, na=False) for k in pattern_map.keys()],
        [f'background-color: {v}' for v in pattern_map.values()],
        default=default
    ).astype(str)


df.style.apply(color_col,
               pattern_map={'Categorical': 'blue',
                            'Quantitative': 'red',
                            'Ordinal': 'green'},
               subset=['Information Type'])





NameError: name 'thisPanel' is not defined

In [2]:
import panel as pn
pn.extension()
thisPanel.view().show()

c:\Actuarial Tools\Python-Based Models\Visualization Tool\.venv\lib\site-packages\panel\template\base.py:121: ParamFutureWarning: Parameter 'busy_indicator' on <class 'panel.template.base.BasicTemplate'> is being given a valid parameter reference BooleanStatus(value=True) but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference BooleanStatus(value=True) will be resolved to its underlying value True. Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or not.
  super().__init__(**{


ValueError: ClassSelector parameter 'FastListTemplate.modal' value must be an instance of ListLike, not Modal(height=0, objects=[Column
    [0] M...], sizing_mode='fixed', width=0).

In [ ]:
thisPanel.template.modal.__getattribute__

AttributeError: 'ListLike' object has no attribute '__bool__'

In [ ]:
import pandas as pd
test=pd.DataFrame(columns=)


In [8]:
import polars as pl
collist=['a','b']
test=pl.DataFrame(schema=collist).with_columns(pl.all().cast(pl.Utf8))


In [9]:
test

a,b
str,str


In [ ]:
for key in thisPanel.analysis.preppedspecs['Source Data Detail']:
    print(key)
    print(thisPanel.analysis.preppedspecs[key])

In [4]:
import _chartClasses as cc
cctest=cc.CreateChart(thisPanel,sourceDataCodeDict=thisPanel.analysis.preppedspecs['Source Data Detail'])

updating data
made it here
callbackDFStructure
and here
transforming data
updatechart
updating data
made it here
and here
transforming data
updatechart
createoptionsdialog


In [5]:
test=thisPanel.analysis.preppedspecs['Source Data Formats'].to_pandas()

In [14]:
test

,Dataset Name,Column,Information Type,Format,Include
0,Summary UW Statistics,Scenario,Categorical,Utf8,Include
1,Summary UW Statistics,Strategy,Categorical,Utf8,Include
2,Summary UW Statistics,Segmentation,Categorical,Utf8,Include
3,Summary UW Statistics,Segment,Categorical,Utf8,Include
4,Summary UW Statistics,Treaty,Categorical,Utf8,Include
5,Summary UW Statistics,Metric,Categorical,Utf8,Include
6,Summary UW Statistics,Statistic,Categorical,Utf8,Include
7,Summary UW Statistics,Value,Quantitave,Float64,Include
8,Summary UW Statistics,Percentile,Ordinal,Float64,Include
9,Summary UW Statistics,Rank,Ordinal,Float64RoundToInt,Include


In [6]:
tabulator_editors = {
    'Dataset Name': None,
    'Column': None,
    'Information Type': {'type': 'list', 'values':['Categorical','Quantitative','Ordinal']},
    'Format': {'type': 'list', 'values':['Utf8','Utf8Trim.0','Float64RoundToInt','Float64','Boolean','Date']},
    'Include': {'type': 'list', 'values':['Include','Exclude']},
}

tab=pn.widgets.Tabulator(test,editors=tabulator_editors,show_index=False)

def color_infotype(s):
    return ['background-color: yellow' if v else '' for v in is_max]

styled.style.map(color_negative_red).apply(highlight_max)



tab.style.background_gradient(cmap="RdYlGn_r", subset=["Information Type"])
tab.show()

NameError: name 'styled' is not defined

In [3]:
pn.widgets.MultiSelect(name='MultiSelect', value=[1,2],
    options=[1,2,3,4], size=8).show()

Launching server at http://localhost:50298


In [4]:
thisPanel.dataformWidgetBlanksDict

{}

In [7]:
template = pn.template.FastListTemplate(
    title='Hi',

).show()

Launching server at http://localhost:52260


In [31]:
import altair as alt
import panel as pn
pn.extension('vega')
alt.data_transformers.enable("vegafusion")

kpi_results_df = thisPanel.analysis.getResults("All","KPI Results").to_pandas()
display(kpi_results_df.head(5))

BokehModel(combine_events=True, render_bundle={'docs_json': {'ad408d25-8b28-42b6-92fe-2368f68c3cd8': {'version…

AttributeError: 'NoneType' object has no attribute 'getResults'

In [31]:
import importlib
import _chartClasses 
import pandas as pd
import matplotlib
matplotlib.use('agg')
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=kpi_results_df,categorycols=['Scenario','Strategy','Segmentation','Segment','KPI Metric','Return Period'],
            chart_type='Heatmap',
            dffiltercols=['Scenario','Segmentation','Segment'],
            legendcols=['Strategy','KPI Metric','Return Period'],
            valuecols=['Value','Rank'],
            #x_valueCols=['Rank'],
            colCombinations={'KPI Metric':['KPI Metric','Return Period']},
            groupByLegendColumn=False).view.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'5fd78b5d-6911-4314-b7b4-58d74518dfab': {'version…

result_x
(52, 4)
result_y
(0, 0)
result_z
(0, 0)
2
USER CHART TYPE UPDATED
Column
_y0  None
_y2  None
data shape (52, 4)
result_x
(52, 4)
result_y
(0, 0)
result_z
(0, 0)
2
USER CHART TYPE UPDATED
Column
_y0  None
_y2  None
data shape (52, 4)
USER CHART TYPE UPDATED
Column
_y0  None
_y2  None
Launching server at http://localhost:62417


In [28]:
import altair as alt
import panel as pn
import pandas as pd
import mercury as mr
df = thisPanel.analysis.getResults("All","KPI Results").to_pandas().fillna(0)
df = df[(df['Segmentation'] == 'All') & (df['Segment'] == 'All')]
print(df.shape)
df.head(5)

2024-05-30 16:35:01,852 — Analysis — DEBUG — Enter getResults:KPI Results All
2024-05-30 16:35:01,865 — Functions — DEBUG — Start GetOrCreateDataFrameIfNotExists
2024-05-30 16:35:01,882 — Analysis — DEBUG — Exit getResults:KPI Results All
(234, 8)


,Scenario,Strategy,Segmentation,Segment,KPI Metric,Percentile,Value,Rank
0,Mitsui,Option 1A,All,All,Gross Loss,0.0,4.245966e+06,9
1,Mitsui,Option 2A,All,All,Gross Loss,0.0,4.245966e+06,3
2,Mitsui,Option 2B,All,All,Gross Loss,0.0,4.245966e+06,1
3,Mitsui,Option 1B,All,All,Gross Loss,0.0,4.245966e+06,8
4,Mitsui,Option 2C,All,All,Gross Loss,0.0,4.245966e+06,5


In [ ]:
#WORKING VERSION OF ALTAIR HEATMAP + BAR AND BUMP CHARTS
def get_corr_df(df):
    data = pd.DataFrame()
    for strategy in sorted(df['Strategy'].unique().tolist()):
        _df = df[df['Strategy'] == strategy]
        data[strategy] = _df['Value'].tolist()
    return data.corr().stack().reset_index().rename(columns={0: 'correlation'})

select_x = alt.selection_point(fields=['level_0'], name='select_x', value='Option 1A')
select_y = alt.selection_point(fields=['level_1'], name='select_y', value='Option 2A')

heatmap = alt.Chart(
    get_corr_df(df),
    title='Click a tile to compare timeseries',
    height=250,
    width=250,
).mark_rect().encode(
    alt.X('level_0').title(None),
    alt.Y('level_1').title(None),
    alt.Color('correlation').scale(domain=[0.999, 1], scheme='rainbow'),
    opacity=alt.condition(select_x & select_y, alt.value(1), alt.value(0.4))
).add_params(
    select_x, select_y
)


#Bar chart comparing the two strategies by selected metrics
base = alt.Chart(
    df[['Strategy','KPI Metric','Value','Rank']],
    height=100,
    width=300,
    title='Bar Chart by Metric'
)

bar = base.transform_filter(
    'indexof(datum.Strategy, select_x.level_0) !== -1'
   '| indexof(datum.Strategy, select_y.level_1) !== -1'
).mark_bar().encode(
    alt.X('KPI Metric:N', axis=alt.Axis(labelAngle=-45)).title(None),
    alt.Y('Value:Q'),
    alt.Color('Strategy:N').legend(orient='top').scale(scheme='paired')
)

bump = base.transform_filter(
    'indexof(datum.Strategy, select_x.level_0) !== -1'
   '| indexof(datum.Strategy, select_y.level_1) !== -1'
).mark_line(point=True).encode(
    alt.X('KPI Metric:N', axis=alt.Axis(labelAngle=-45)),
    alt.Y('Rank:O', axis=alt.Axis(grid=True)),
    color = alt.Color("Strategy:N")
).transform_window(
    rank='rank()',
    sort=[alt.SortField("Rank", order="descending")],
    groupby=['KPI Metric:N']
).properties(
    width = 300,
    height = 100,
    title="Bump Chart for Strategies"
)
output = (bar & bump) | heatmap
output
# vega_spec = output.to_dict(format="vega")
# pn.pane.Vega(output.to_dict(format='vega'))

In [ ]:
import pandas as pd
import numpy as np
import altair as alt


# Create timeseries data
rng = np.random.default_rng(905)
ex_ts = pd.DataFrame(
    rng.random((10, 4)),
    columns=['a', 'b', 'c', 'd'],
).assign(
    date=pd.date_range(
        start=pd.to_datetime('2022-02-22')-pd.Timedelta(9, unit='D'),
        end=pd.to_datetime('2022-02-22')).strftime('%Y-%m-%d'),
)

# Create heatmap with selection
select_x = alt.selection_point(fields=['level_0'], name='select_x', value='b')
select_y = alt.selection_point(fields=['level_1'], name='select_y', value='d')
heatmap = alt.Chart(
    ex_ts.drop(columns='date').corr().stack().reset_index().rename(columns={0: 'correlation'}),
    title='Click a tile to compare timeseries',
    height=250,
    width=250,
).mark_rect().encode(
    alt.X('level_0').title(None),
    alt.Y('level_1').title(None),
    alt.Color('correlation').scale(domain=[-1, 1], scheme='blueorange'),
    opacity=alt.condition(select_x & select_y, alt.value(1), alt.value(0.4))
).add_params(
    select_x, select_y
)

# Create chart with individual lines/timeseries
base = alt.Chart(
    ex_ts.melt(
        id_vars='date',
        var_name='category',
        value_name='value',
    ),
    height=100,
    width=300,
    title='Individual timeseries',
)
lines = base.transform_filter(
    # If the category is not in the selected values, the returned index is -1
    'indexof(datum.category, select_x.level_0) !== -1'
   '| indexof(datum.category, select_y.level_1) !== -1'
).mark_line().encode(
    alt.X('date:T').axis(labels=False).title(None),
    alt.Y('value').scale(domain=(0, 1)),
    alt.Color('category').legend(orient='top', offset=-20).title(None)
)

# Create chart with difference between lines/timeseries
dynamic_title = alt.Title(alt.expr(f'"Difference " + {select_x.name}.level_0 + " - " + {select_y.name}.level_1'))
# We pivot transform to get each category as a column
lines_diff = base.transform_pivot(
    'category', 'value', groupby=['date']
# In the calculate transform we use the values from the selection to subset the columns to substract
).transform_calculate(
    difference = f'datum[{select_x.name}.level_0] - datum[{select_y.name}.level_1]'
).mark_line(color='grey').encode(
    alt.X('date:T').axis(format='%Y-%m-%d').title(None),
    alt.Y('difference:Q').scale(domain=(-1, 1)),
).properties(
    title=dynamic_title
)

# Layout the charts
(lines & lines_diff) | heatmap

In [21]:
#1-dimenionsal example
import importlib
import _chartClasses
importlib.reload(_misc)
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=df,
            chart_type='Table-1dim',
            categorycols=['Scenario','Strategy','Segmentation','Segment','KPI Metric','Percentile'],
            dffiltercols=['Scenario','Segmentation','Segment'],
            legendcols=['Strategy','KPI Metric','Percentile'],
            valuecols=['Value','Rank'],
            x_valueCols=['Rank'],
            colCombinations={'KPI Metric':['KPI Metric','Percentile']},
            groupByLegendColumn=True).view.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'8df56bc7-02b6-43f6-8be9-fc919d321402': {'version…

result_x
(234, 4)
result_y
(0, 0)
result_z
(0, 0)
2
USER CHART TYPE UPDATED
Column
data shape (234, 4)
USER CHART TYPE UPDATED
Column
Launching server at http://localhost:62225


In [ ]:
#1-dimenionsal example
import importlib
import _chartClasses
importlib.reload(_misc)
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=df,
            chart_type='Waterfall',
            categorycols=['Scenario','Strategy','Segmentation','Segment','KPI Metric','Percentile'],
            dffiltercols=['Scenario','Segmentation','Segment','Strategy'],
            legendcols=['KPI Metric'],
            valuecols=['Value'],
            groupByLegendColumn=True).view.show()

In [29]:
#1-dimenionsal example
import importlib
import _chartClasses
importlib.reload(_misc)
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=df,
            chart_type='Heatmap',
            categorycols=['Scenario','Strategy','Segmentation','Segment'],
            dffiltercols=['Scenario','Segmentation','Segment'],
            legendcols=['Strategy','KPI Metric'],
            valuecols=['KPI Metric','Value','Rank'],
            groupByLegendColumn=False).view.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'a9c199d8-82ee-4b76-bee7-dc8be40ce313': {'version…

result_x
(52, 5)
result_y
(0, 0)
result_z
(0, 0)
2
USER CHART TYPE UPDATED
Column
_y0  None
_y2  None
data shape (52, 5)


ValueError: Cannot display a pandas.DataFrame with duplicate column names.

In [23]:
#2-dimensional example
import importlib
import _chartClasses
importlib.reload(_misc)
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=df,
            chart_type='Radial Chart-2dim', #Chart type (some are 1-dimensional, some are 2-dimensional)
            categorycols=['Scenario','Strategy','Segmentation','Segment'], #Categorical columns
            dffiltercols=['Scenario','Segmentation','Segment'], #Columns to filter the data
            legendcols=['Strategy'], #Column values to be multichoice select
            valuecols=['Value'], #Value column in the dataset
            x_filterCols={'KPI Metric':['Ceded UW Profit','Ceded Premium']}, #For 2-dimensional data, widget to filter x-axis
            x_valueCols=['Value'], #Value column that x_filterCols filters down
            y_filterCols={'KPI Metric':['Net UW Profit','Net Loss']}, #For 2-dimensional data, widget to filter y-axis
            y_valueCols=['Value'], #Value column that y_filterCols filters down
            groupByLegendColumn=True #Whether or not data is grouped by legend column
            ).view.show()



BokehModel(combine_events=True, render_bundle={'docs_json': {'0a0ca51e-173c-49c0-85bc-53117388a673': {'version…

result_x
(9, 2)
self._y1:  Net UW Profit
result_y
(9, 2)
result_z
(0, 0)
1
USER CHART TYPE UPDATED
Column
data shape (9, 3)
USER CHART TYPE UPDATED
Column
Launching server at http://localhost:62254


In [22]:
#3-dimensional example
import importlib
import _chartClasses
importlib.reload(_misc)
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=df,
            chart_type='Table-3dim', #Chart type (some are 1-dimensional, some are 2-dimensional)
            categorycols=['Scenario','Strategy','Segmentation','Segment'], #Categorical columns
            dffiltercols=['Scenario','Segmentation','Segment'], #Columns to filter the data
            legendcols=['Strategy'], #Column values to be multichoice select
            valuecols=['Value'], #Value column in the dataset
            x_filterCols={'KPI Metric':['Ceded UW Profit','Ceded Premium']}, #For 2-dimensional data, widget to filter x-axis
            x_valueCols=['Value'], #Value column that x_filterCols filters down
            y_filterCols={'KPI Metric':['Net UW Profit','Net Loss']}, #For 2-dimensional data, widget to filter y-axis
            y_valueCols=['Value'], #Value column that y_filterCols filters down
            z_filterCols={'KPI Metric':['Net Loss']}, #For 3-dimensional data, widget to filter z-axis
            z_valueCols=['Value'],
            groupByLegendColumn=True #Whether or not data is grouped by legend column
            ).view.show()



BokehModel(combine_events=True, render_bundle={'docs_json': {'95463eae-78e1-4b85-8af6-e6bcefc52964': {'version…

result_x
(9, 2)
self._y1:  Net UW Profit
result_y
(9, 2)
result_z
(9, 2)
USER CHART TYPE UPDATED
Column
data shape (9, 4)
USER CHART TYPE UPDATED
Column
Launching server at http://localhost:62241


In [ ]:
#2-dimensional example
import importlib
import _chartClasses
importlib.reload(_misc)
importlib.reload(_chartClasses)
_chartClasses.CreateChart(source_df=df,
            chart_type='Table-2dim', #Chart type (some are 1-dimensional, some are 2-dimensional)
            categorycols=['Scenario','Strategy','Segmentation','Segment'], #Categorical columns
            dffiltercols=['Scenario','Segmentation','Segment'], #Columns to filter the data
            legendcols=['Strategy'], #Column values to be multichoice select
            valuecols=['Value'], #Value column in the dataset
            x_filterCols={'KPI Metric':['Ceded UW Profit','Ceded Premium']}, #For 2-dimensional data, widget to filter x-axis
            x_valueCols=['Value'], #Value column that x_filterCols filters down
            y_filterCols={'KPI Metric':['Net UW Profit','Net Loss']}, #For 2-dimensional data, widget to filter y-axis
            y_valueCols=['Value'], #Value column that y_filterCols filters down
            # z_filterCols={'KPI Metric':['Net Loss']}, #For 3-dimensional data, widget to filter z-axis
            # z_valueCols=['Value'],
            groupByLegendColumn=True #Whether or not data is grouped by legend column
            ).view.show()

